<a href="https://colab.research.google.com/github/akari2951/Telco-customer-churn/blob/main/Animal_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gspread gspread_dataframe oauth2client
!pip install pandas requests

In [3]:
# 認証
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import requests
import time
import json
from gspread_dataframe import set_with_dataframe

# Googleスプレッドシートに接続する関数
def connect_to_google_sheets():
    creds, _ = default()
    gc = gspread.authorize(creds)
    return gc

# 入力データを読み込む関数
def read_animal_names(gc, spreadsheet_name, column_name="animal_name"):
    worksheet = gc.open(spreadsheet_name).sheet1
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)

    if column_name not in df.columns:
        raise ValueError(f"スプレッドシートに『{column_name}』という列が必要です。")

    return df

# サジェスト取得関数
def get_google_suggest(keyword):
    url = f"https://suggestqueries.google.com/complete/search?client=firefox&q={keyword}"
    r = requests.get(url)
    return r.json()[1] if r.status_code == 200 else []

# 動物名ごとにサジェスト取得して結果をまとめる関数
def collect_suggestions(df, column_name="animal_name"):
    results = []
    for name in df[column_name]:
        print(f"取得中: {name}")
        google = get_google_suggest(name)
        results.append({
            '動物名': name,
            'Googleサジェスト': "\n".join(google),
        })
        time.sleep(1.0)  # 負荷軽減
    return pd.DataFrame(results)


# 結果を書き込む関数
def write_results_to_sheet(gc, spreadsheet_name, result_df, sheet_name="結果"):
    spreadsheet = gc.open(spreadsheet_name)

    # 既存シートがある場合は削除
    try:
        existing = spreadsheet.worksheet(sheet_name)
        spreadsheet.del_worksheet(existing)
    except:
        pass

    # 新しいシート作成して書き込み
    output_sheet = spreadsheet.add_worksheet(title=sheet_name, rows="1000", cols="10")
    set_with_dataframe(output_sheet, result_df)


# メイン処理
def main():
    sheet_name = "Exotic_animal"
    gc = connect_to_google_sheets()
    df = read_animal_names(gc, sheet_name)
    result_df = collect_suggestions(df)
    write_results_to_sheet(gc, sheet_name, result_df)
    print("サジェストの取得と結果シートへの書き込みが完了しました！")

if __name__ == "__main__":
  main()

取得中: チンチラ
取得中: フクロモモンガ
取得中: シマリス
取得中: フェレット
取得中: モルモット
取得中: ウサギ
取得中: ゴールデンハムスター
取得中: リスザル
取得中: セキセイインコ
取得中: デグー
取得中: フェネック
取得中: ショウガラゴ
取得中: オカメインコ
取得中: ハリネズミ
取得中: ヒョウモントカゲモドキ
サジェストの取得と結果シートへの書き込みが完了しました！
